In [245]:
import pandas as pd
import category_encoders as ce
from sklearn import preprocessing
from  sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif
  
import seaborn as sns

In [246]:
Train_data =pd.read_csv("C:/Users/ahmed/Downloads/AI for CS/Assignment 2/traindata.csv")
X_test=pd.read_csv("C:/Users/ahmed/Downloads/AI for CS/Assignment 2/testdata.csv")

In [247]:
Train_data.head(5)

ID  duration protocol_type service flag  src_bytes  dst_bytes  land  \
0   0         0           tcp    http   SF        295       2584     0   
1   1         0           tcp    http   SF        312        839     0   
2   2         0           tcp     ftp   SF       1225       2451     0   
3   3         0           tcp    http   SF        293        296     0   
4   4         0           tcp     X11  REJ          0          0     0   

   wrong_fragment  urgent  ...  dst_host_same_srv_rate  \
0               0       0  ...                    1.00   
1               0       0  ...                    1.00   
2               0       0  ...                    0.45   
3               0       0  ...                    1.00   
4               0       0  ...                    0.01   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.00                         0.01   
1                    0.00                         0.00   
2                    0.02                         0.00   
3                    0.00                         0.00   
4                    0.03                         0.01   

   dst_host_srv_diff_host_rate  dst_host_serror_rate  \
0                         0.03                   0.0   
1                         0.02                   0.0   
2                         0.00                   0.0   
3                         0.02                   0.0   
4                         0.00                   0.0   

   dst_host_srv_serror_rate  dst_host_rerror_rate  dst_host_srv_rerror_rate  \
0                       0.0                  0.00                      0.01   
1                       0.0                  0.01                      0.01   
2                       0.0                  0.00                      0.00   
3                       0.0                  0.00                      0.02   
4                       0.0                  0.01                      1.00   

   level  Class  
0     21      0  
1     21      0  
2     11      1  
3     21      0  
4     11      0  

[5 rows x 44 columns]

In [248]:
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
Train_data['protocol_type']= label_encoder.fit_transform(Train_data['protocol_type'])
X_test['protocol_type']= label_encoder.transform(X_test['protocol_type'])


In [249]:
Train_data['service'] = Train_data['service'].astype(str)
X_test['service'] = X_test['service'].astype(str)
label_encoder = preprocessing.LabelEncoder()
  
  
combined_data = pd.concat([Train_data['service'], X_test['service']], axis=0)
label_encoder.fit(combined_data)
Train_data['service']= label_encoder.transform(Train_data['service'])

X_test['service'] = label_encoder.transform(X_test['service'])



In [250]:
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
Train_data['flag']= label_encoder.fit_transform(Train_data['flag'])
X_test['flag']= label_encoder.transform(X_test['flag'])

In [251]:
y=Train_data["Class"]
x=Train_data.iloc[:, :-1]

In [252]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [236]:
X_train.shape

(18401, 43)

In [237]:
RF_model=RandomForestClassifier(random_state=42,max_depth=4,min_samples_leaf=4,max_features=0.8,oob_score=True)
RF_model.fit(X_train,y_train)
RF_y_pred=RF_model.predict(X_val)


In [238]:
def Cal_F1_scoure(y_test,y_Test_pred):
    cm = confusion_matrix(y_test,y_Test_pred)
    true_positive = cm[1, 1]
    false_positive = cm[0, 1]
    false_negative = cm[1, 0]
    true_negative = cm[0, 0]
    
    return (2 * true_positive / (2 * true_positive + false_positive + false_negative))

In [239]:
RF_y_pred_train=RF_model.predict(X_train)

In [240]:
print(Cal_F1_scoure(y_val,RF_y_pred))

0.9906199954243881


In [124]:
print(Cal_F1_scoure(y_train, RF_y_pred_train))

0.9898194499128184


In [127]:
Cat_model = CatBoostClassifier(random_seed=42 )
Cat_model.fit(X_train,y_train)
Cat_y_pred=Cat_model.predict(X_val)

Learning rate set to 0.035729
0:	learn: 0.5913577	total: 11.6ms	remaining: 11.6s
1:	learn: 0.5102206	total: 22.5ms	remaining: 11.2s
2:	learn: 0.4305470	total: 32.5ms	remaining: 10.8s
3:	learn: 0.3621050	total: 41.8ms	remaining: 10.4s
4:	learn: 0.3211033	total: 51.4ms	remaining: 10.2s
5:	learn: 0.2793233	total: 60ms	remaining: 9.95s
6:	learn: 0.2403080	total: 70ms	remaining: 9.94s
7:	learn: 0.2124400	total: 79.9ms	remaining: 9.9s
8:	learn: 0.1863998	total: 91.1ms	remaining: 10s
9:	learn: 0.1600119	total: 103ms	remaining: 10.2s
10:	learn: 0.1385959	total: 113ms	remaining: 10.2s
11:	learn: 0.1204045	total: 125ms	remaining: 10.3s
12:	learn: 0.1089014	total: 137ms	remaining: 10.4s
13:	learn: 0.0973294	total: 149ms	remaining: 10.5s
14:	learn: 0.0872454	total: 163ms	remaining: 10.7s
15:	learn: 0.0792139	total: 175ms	remaining: 10.7s
16:	learn: 0.0746930	total: 186ms	remaining: 10.8s
17:	learn: 0.0671141	total: 199ms	remaining: 10.8s
18:	learn: 0.0621969	total: 211ms	remaining: 10.9s
19:	learn

In [128]:
Cat_y_pred_train=Cat_model.predict(X_train)

In [129]:
print(Cal_F1_scoure(y_val,Cat_y_pred))

0.9995395948434622


In [130]:
print(Cal_F1_scoure(y_train, Cat_y_pred_train))

1.0


In [253]:
def select_feature(X_train, y_train,X_validation , FSM):
    fs = FSM
    fs.fit(X_train, y_train)
    X_train_new = fs.transform(X_train)

    X_validation_new = fs.transform(X_validation)
    
    feature_indices = fs.get_support(indices=True)
   

    return X_train_new,X_validation_new,feature_indices

In [254]:
acc_dict_DT_validation = {}
acc_dict__DT_Test = {}

acc_dict_KNN_validation = {}
acc_dict_KNN_Test = {}
feature_indices_dict={}

In [255]:
RF_model=CatBoostClassifier(random_seed=42 )

In [256]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [257]:
for k in range(1, 28):

    # Filter method: Information Gain
    fsm = SelectKBest(score_func=f_classif, k=k)
    
    X_train_new_SBest,X_validation_new_SBest,feature_indices = select_feature(X_train, y_train, X_val, fsm)
    feature_indices_dict[k]=feature_indices
    

    # tree
    
    RF_model.fit(X_train_new_SBest, y_train)
    y_validation_pred_DT = RF_model.predict(X_validation_new_SBest)
    acc_dict_DT_validation[k]=Cal_F1_scoure(y_val,y_validation_pred_DT)
    
    

C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [20 21] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Learning rate set to 0.035729
0:	learn: 0.6625114	total: 14.4ms	remaining: 14.4s
1:	learn: 0.6350507	total: 25.7ms	remaining: 12.8s
2:	learn: 0.6093268	total: 48.1ms	remaining: 16s
3:	learn: 0.5850235	total: 72.8ms	remaining: 18.1s
4:	learn: 0.5628177	total: 102ms	remaining: 20.3s
5:	learn: 0.5433268	total: 116ms	remaining: 19.1s
6:	learn: 0.5257942	total: 128ms	remaining: 18.1s
7:	learn: 0.5111053	total: 149ms	remaining: 18.5s
8:	learn: 0.4978913	total: 165ms	remaining: 18.2s
9:	learn: 0.4877581	total: 175ms	remaining: 17.3s
10:	learn: 0.4753930	total: 236ms	remaining: 21.2s
11:	learn: 0.4657555	total: 247ms	remaining: 20.3s
12:	learn: 0.4569521	total: 255ms	remaining: 19.4s
13:	learn: 0.4477468	total: 269ms	remaining: 19s
14:	learn: 0.4405678	total: 280ms	remaining: 18.4s
15:	learn: 0.4328533	total: 292ms	remaining: 18s
16:	learn: 0.4260448	total: 305ms	remaining: 17.6s
17:	learn: 0.4203798	total: 318ms	remaining: 17.4s
18:	learn: 0.4152714	total: 330ms	remaining: 17s
19:	learn: 0.41

C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [20 21] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


2:	learn: 0.5814638	total: 29.8ms	remaining: 9.9s
3:	learn: 0.5513340	total: 39.3ms	remaining: 9.79s
4:	learn: 0.5265403	total: 49.2ms	remaining: 9.79s
5:	learn: 0.5040471	total: 59.2ms	remaining: 9.8s
6:	learn: 0.4814191	total: 68.1ms	remaining: 9.66s
7:	learn: 0.4632211	total: 78.8ms	remaining: 9.77s
8:	learn: 0.4446068	total: 89.6ms	remaining: 9.87s
9:	learn: 0.4303701	total: 99.5ms	remaining: 9.85s
10:	learn: 0.4159467	total: 110ms	remaining: 9.89s
11:	learn: 0.4020544	total: 121ms	remaining: 9.95s
12:	learn: 0.3912189	total: 129ms	remaining: 9.82s
13:	learn: 0.3808794	total: 140ms	remaining: 9.86s
14:	learn: 0.3725682	total: 150ms	remaining: 9.85s
15:	learn: 0.3631113	total: 160ms	remaining: 9.83s
16:	learn: 0.3549176	total: 170ms	remaining: 9.85s
17:	learn: 0.3480134	total: 180ms	remaining: 9.82s
18:	learn: 0.3410566	total: 188ms	remaining: 9.7s
19:	learn: 0.3352714	total: 199ms	remaining: 9.74s
20:	learn: 0.3304005	total: 207ms	remaining: 9.66s
21:	learn: 0.3264662	total: 217ms	

C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [20 21] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


...

Learning rate set to 0.035729
0:	learn: 0.5979625	total: 9.47ms	remaining: 9.46s
1:	learn: 0.5140812	total: 19ms	remaining: 9.5s
2:	learn: 0.4421658	total: 28.3ms	remaining: 9.41s
3:	learn: 0.3882767	total: 36.6ms	remaining: 9.13s
4:	learn: 0.3445813	total: 46.3ms	remaining: 9.22s
5:	learn: 0.2978408	total: 56.4ms	remaining: 9.35s
6:	learn: 0.2589260	total: 65.7ms	remaining: 9.31s
7:	learn: 0.2279650	total: 76.1ms	remaining: 9.43s
8:	learn: 0.2019438	total: 84.7ms	remaining: 9.32s
9:	learn: 0.1825873	total: 94ms	remaining: 9.3s
10:	learn: 0.1622126	total: 103ms	remaining: 9.3s
11:	learn: 0.1476121	total: 112ms	remaining: 9.26s
12:	learn: 0.1385314	total: 121ms	remaining: 9.21s
13:	learn: 0.1268061	total: 131ms	remaining: 9.21s
14:	learn: 0.1169080	total: 141ms	remaining: 9.29s
15:	learn: 0.1084910	total: 152ms	remaining: 9.33s
16:	learn: 0.0999216	total: 163ms	remaining: 9.45s
17:	learn: 0.0932094	total: 174ms	remaining: 9.49s
18:	learn: 0.0879585	total: 186ms	remaining: 9.6s
19:	learn

C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [20 21] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Learning rate set to 0.035729
0:	learn: 0.5893745	total: 14.1ms	remaining: 14s
1:	learn: 0.5046263	total: 27.3ms	remaining: 13.6s
2:	learn: 0.4368389	total: 39.2ms	remaining: 13s
3:	learn: 0.3748264	total: 49.7ms	remaining: 12.4s
4:	learn: 0.3268339	total: 61.5ms	remaining: 12.2s
5:	learn: 0.2835520	total: 73.7ms	remaining: 12.2s
6:	learn: 0.2455525	total: 84.2ms	remaining: 11.9s
7:	learn: 0.2172837	total: 97.1ms	remaining: 12s
8:	learn: 0.1992208	total: 109ms	remaining: 12s
9:	learn: 0.1745916	total: 122ms	remaining: 12.1s
10:	learn: 0.1552109	total: 134ms	remaining: 12.1s
11:	learn: 0.1411697	total: 147ms	remaining: 12.1s
12:	learn: 0.1285497	total: 160ms	remaining: 12.1s
13:	learn: 0.1186522	total: 172ms	remaining: 12.1s
14:	learn: 0.1099767	total: 182ms	remaining: 11.9s
15:	learn: 0.1017610	total: 194ms	remaining: 11.9s
16:	learn: 0.0950843	total: 206ms	remaining: 11.9s
17:	learn: 0.0900142	total: 218ms	remaining: 11.9s
18:	learn: 0.0844946	total: 232ms	remaining: 12s
19:	learn: 0.

C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [20 21] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\ahmed\anaconda3\envs\My_env\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Learning rate set to 0.035729
0:	learn: 0.5824698	total: 25.7ms	remaining: 25.7s
1:	learn: 0.4900604	total: 37.2ms	remaining: 18.6s
2:	learn: 0.4189719	total: 47.3ms	remaining: 15.7s
3:	learn: 0.3540000	total: 56.7ms	remaining: 14.1s
4:	learn: 0.3072881	total: 67.4ms	remaining: 13.4s
5:	learn: 0.2656690	total: 78.3ms	remaining: 13s
6:	learn: 0.2392109	total: 88.3ms	remaining: 12.5s
7:	learn: 0.2129043	total: 97.4ms	remaining: 12.1s
8:	learn: 0.1901810	total: 108ms	remaining: 11.9s
9:	learn: 0.1684060	total: 120ms	remaining: 11.9s
10:	learn: 0.1511929	total: 138ms	remaining: 12.4s
11:	learn: 0.1378423	total: 161ms	remaining: 13.3s
12:	learn: 0.1242257	total: 178ms	remaining: 13.5s
13:	learn: 0.1139558	total: 189ms	remaining: 13.3s
14:	learn: 0.1044914	total: 201ms	remaining: 13.2s
15:	learn: 0.0990464	total: 212ms	remaining: 13.1s
16:	learn: 0.0936767	total: 224ms	remaining: 13s
17:	learn: 0.0894992	total: 237ms	remaining: 13s
18:	learn: 0.0831163	total: 250ms	remaining: 12.9s
19:	learn

In [224]:
def Draw_plots(acc_list,baseline_accuracy, Title=None):
    fig = plt.figure()  # Create a new figure
    plt.plot(*zip(*sorted(acc_list.items())))
    plt.axhline(y=baseline_accuracy, color='r', linestyle='--', label=f'Baseline Accuracy {baseline_accuracy:.3f}')
    plt.legend()
    # Title = "Feature Selection with Information Gain Method"
    plt.title(Title, fontsize=16)
    plt.xlabel("Number of Features", fontsize=16)
    plt.ylabel("Accuracy", fontsize=16)
    plt.show()
    print("Maximum accuracy:", max(acc_list.values()))
    print("Best number of features:", max(acc_list, key=acc_list.get))
    plt.show()
    return   max(acc_list, key=acc_list.get)
    # plt.close(fig)

In [165]:
Max_DT_test=Draw_plots(acc_dict_DT_validation,0,"Feature Selection with Information Gain Method With DecisionTr")

Maximum accuracy: 0.9995395948434622
Best number of features: 27


In [258]:
def find_list_with_length(feature_indices_dict,desired_length,Data):
   
    new_features = feature_indices_dict.get(desired_length, None)
    if new_features is not None:
        new_X_train = Data.iloc[:, new_features]
        return new_X_train
    else:
        # Handle the case when the desired_length is not found in the dictionary
        print(f"No list with length {desired_length} found in the dictionary.")
        return None

In [259]:
X_train_new=find_list_with_length(feature_indices_dict,Max_DT_test,X_train)

X_Validation_new=find_list_with_length(feature_indices_dict,Max_DT_test,X_val)
X_test_new=find_list_with_length(feature_indices_dict,Max_DT_test,X_test)

X_Validation_new

duration  protocol_type  service  flag  wrong_fragment  logged_in  \
14905         0              2       11     9               0          0   
3470          0              1       22     6               0          1   
16816         0              1       51     9               0          1   
7660          0              1       22     9               0          1   
12063         0              1       46     5               0          0   
...         ...            ...      ...   ...             ...        ...   
5601         10              1       51     9               0          1   
16861         0              1       41     1               0          0   
10254         0              1       22     9               0          1   
12837         0              1       44     2               0          0   
21643         0              1       19     9               0          1   

       num_access_files  is_guest_login  count  serror_rate  ...  \
14905                 0               0    112         0.00  ...   
3470                  0               0      2         0.50  ...   
16816                 0               0      1         0.00  ...   
7660                  0               0     24         0.00  ...   
12063                 0               0    111         1.00  ...   
...                 ...             ...    ...          ...  ...   
5601                  0               0      1         0.00  ...   
16861                 0               0    510         0.13  ...   
10254                 0               0     10         0.00  ...   
12837                 0               0    162         0.00  ...   
21643                 0               0     11         0.00  ...   

       dst_host_srv_count  dst_host_same_srv_rate  dst_host_diff_srv_rate  \
14905                 223                    0.87                    0.01   
3470                  255                    1.00                    0.00   
16816                 203                    0.79                    0.04   
7660                  255                    1.00                    0.00   
12063                   6                    0.50                    0.33   
...                   ...                     ...                     ...   
5601                  142                    0.56                    0.03   
16861                   1                    0.00                    1.00   
10254                 255                    1.00                    0.00   
12837                  18                    0.07                    0.07   
21643                  21                    0.08                    0.06   

       dst_host_same_src_port_rate  dst_host_srv_diff_host_rate  \
14905                         0.00                         0.00   
3470                          0.50                         0.02   
16816                         0.01                         0.01   
7660                          0.00                         0.00   
12063                         0.08                         0.00   
...                            ...                          ...   
5601                          0.00                         0.00   
16861                         0.00                         0.00   
10254                         0.01                         0.02   
12837                         0.00                         0.00   
21643                         0.08                         0.00   

       dst_host_serror_rate  dst_host_srv_serror_rate  dst_host_rerror_rate  \
14905                  0.00                      0.00                  0.00   
3470                   0.50                      0.01                  0.00   
16816                  0.00                      0.00                  0.00   
7660                   0.00                      0.00                  0.00   
12063                  1.00                      1.00                  0.00   
...                     ...                       ...                   .

In [1]:
Cat_model = CatBoostClassifier(random_seed=42 )
Cat_model.fit(X_train_new,y_train)
RF_y_pred=Cat_model.predict(X_Validation_new)

Cat_y_pred=Cat_model.predict(X_test_new)

NameError: name 'CatBoostClassifier' is not defined

In [261]:
Cat_y_pred_train=RF_model.predict(X_train_new)
print(Cal_F1_scoure(y_train, Cat_y_pred_train))
print(Cal_F1_scoure(y_val,RF_y_pred))

0.9997168582592446
0.9967830882352942


In [263]:
OUTPUT_sample=pd.DataFrame({"Class":Cat_y_pred})
OUTPUT_sample=OUTPUT_sample.rename_axis("ID")
OUTPUT_sample
OUTPUT_sample.to_csv("sampleSubmissionCat_filter.csv")

In [211]:

model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='Logloss')
model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=50)


0:	learn: 0.4562899	test: 0.4552821	best: 0.4552821 (0)	total: 11.1ms	remaining: 11.1s
50:	learn: 0.0054922	test: 0.0069476	best: 0.0069476 (50)	total: 668ms	remaining: 12.4s
100:	learn: 0.0023655	test: 0.0043868	best: 0.0043868 (100)	total: 1.08s	remaining: 9.64s
150:	learn: 0.0018094	test: 0.0039812	best: 0.0039798 (139)	total: 1.58s	remaining: 8.9s
200:	learn: 0.0017318	test: 0.0039060	best: 0.0039060 (200)	total: 2.07s	remaining: 8.25s
250:	learn: 0.0017315	test: 0.0039062	best: 0.0039060 (203)	total: 2.52s	remaining: 7.54s
300:	learn: 0.0016995	test: 0.0038798	best: 0.0038792 (279)	total: 2.94s	remaining: 6.82s
350:	learn: 0.0016990	test: 0.0038808	best: 0.0038792 (279)	total: 3.32s	remaining: 6.13s
400:	learn: 0.0016986	test: 0.0038813	best: 0.0038792 (279)	total: 3.76s	remaining: 5.61s
450:	learn: 0.0016983	test: 0.0038818	best: 0.0038792 (279)	total: 4.19s	remaining: 5.1s
500:	learn: 0.0016980	test: 0.0038821	best: 0.0038792 (279)	total: 4.6s	remaining: 4.58s
550:	learn: 0.0016

In [212]:
# Get feature importances
feature_importance = model.get_feature_importance(prettified=True)
print(feature_importance)


                     Feature Id   Importances
0                     src_bytes  3.217868e+01
1                 protocol_type  1.117753e+01
2                         level  9.315697e+00
3                     srv_count  6.632153e+00
4                     dst_bytes  6.042149e+00
5                         count  4.340189e+00
6                dst_host_count  3.551448e+00
7      dst_host_srv_serror_rate  3.227922e+00
8        dst_host_same_srv_rate  2.875031e+00
9                       service  2.643245e+00
10       dst_host_diff_srv_rate  2.431269e+00
11           dst_host_srv_count  2.079148e+00
12                    logged_in  2.042876e+00
13  dst_host_same_src_port_rate  1.888339e+00
14         dst_host_serror_rate  1.473974e+00
15                same_srv_rate  1.390612e+00
16              srv_serror_rate  9.963631e-01
17                  serror_rate  9.551783e-01
18                         flag  9.318201e-01
19                          hot  7.230042e-01
20         dst_host_rerror_rate  6

In [213]:
# Select top N features
top_n_features = feature_importance.head(15)['Feature Id'].tolist()

# Select features with importance score above a threshold
threshold = 0.1  # Adjust the threshold as needed
important_features = feature_importance[feature_importance['Importances'] > threshold]['Feature Id'].tolist()


In [214]:
important_features

['src_bytes',
 'protocol_type',
 'level',
 'srv_count',
 'dst_bytes',
 'count',
 'dst_host_count',
 'dst_host_srv_serror_rate',
 'dst_host_same_srv_rate',
 'service',
 'dst_host_diff_srv_rate',
 'dst_host_srv_count',
 'logged_in',
 'dst_host_same_src_port_rate',
 'dst_host_serror_rate',
 'same_srv_rate',
 'srv_serror_rate',
 'serror_rate',
 'flag',
 'hot',
 'dst_host_rerror_rate',
 'wrong_fragment',
 'dst_host_srv_diff_host_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'diff_srv_rate',
 'num_compromised',
 'dst_host_srv_rerror_rate']

In [215]:
# Subset the data to include only important features
X_train_selected = X_train[important_features]
X_val_selected = X_val[important_features]
X_test_selected = X_test[important_features]

# Initialize and train the CatBoost model with selected features
model_selected_features = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='Logloss')
model_selected_features.fit(X_train_selected, y_train, eval_set=(X_val_selected, y_val), verbose=50)


0:	learn: 0.4232732	test: 0.4229245	best: 0.4229245 (0)	total: 8.16ms	remaining: 808ms
50:	learn: 0.0051399	test: 0.0066926	best: 0.0066926 (50)	total: 494ms	remaining: 475ms
99:	learn: 0.0022132	test: 0.0042535	best: 0.0042535 (99)	total: 4.14s	remaining: 0us

bestTest = 0.0042534526
bestIteration = 99


In [216]:
Cat_new_y_pred=model_selected_features.predict(X_test_selected)

In [217]:
OUTPUT_sample=pd.DataFrame({"Class":Cat_new_y_pred})
OUTPUT_sample=OUTPUT_sample.rename_axis("ID")
OUTPUT_sample
OUTPUT_sample.to_csv("sampleSubmissionCat_filter_bult.csv")